In [1]:
from ollama import chat
import nltk
from pydantic import BaseModel

In [2]:
#client = OllamaClient(host="http://localhost:11434")
LLM_model = "granite4:small-h"
#response = client.chat(messages=[system_msg.to_json(), assistant_msg.to_json(), user_msg.to_json()], model="llama3", stream=False)
#print("Content:", response.content)
#print("Parsed data:", response.parsed_data)

In [3]:
result_list = []
the_text = "Standard Retrieval-Augmented Generation (RAG) relies on chunk-based retrieval, whereas GraphRAG advances this approach by graph-based knowledge representation. However, existing graph-based RAG approaches are constrained by binary relations, as each edge in an ordinary graph connects only two entities, limiting their ability to represent the n-ary relations (n >= 2) in real-world knowledge. In this work, we propose HyperGraphRAG, a novel hypergraph-based RAG method that represents n-ary relational facts via hyperedges, and consists of knowledge hypergraph construction, retrieval, and generation. Experiments across medicine, agriculture, computer science, and law demonstrate that HyperGraphRAG outperforms both standard RAG and previous graph-based RAG methods in answer accuracy, retrieval efficiency, and generation quality. Our data and code are publicly available at this https URL."

In [4]:
class topicMap:
    locations: list[int] = []
    text: str = ""
    def __init__(self, text: str, new_locations: list[int]):
        self.text = text
        self.locations = new_locations
    
    def __str__(self) -> str:
        return f"Topic: {self.text}, Locations: {self.locations}"
    
    def __repr__(self) -> str:
        return self.__str__()
    
    def __eq__(self, text: object) -> bool:
        return self.text == text

In [21]:
class Topic(BaseModel):
    label: str
    description: str
    category: str

class Relation(BaseModel):
    source: str
    target: str
    label: str
    weight: float
    description: str
#    category: str
    
class KnowledgeGraph(BaseModel):
    topics: list[Topic]
    relations: list[Relation]

#KnowledgeGraph.model_json_schema()

In [27]:
LLM_model = "glm-4.7-flash"
LLM_model = "glm-4.7-flash:q8_0"
#LLM_model = "qwen3.5:397b-cloud"
#LLM_model = "glm-5:cloud"
system_msg = {"role": "system", "content": """
You are a helpful assistant that extracts topics and relations from a text to create a knowledge graph.

INSTRUCTIONS:
1. Extract all key topics/entities mentioned in the text, with a category and a brief description for each topic
2. Identify meaningful relationships between these topics, with a weight and a brief description for each relationship
3. Use the exact topic labels as they appear in the text for sourceTopic and targetTopic fields
4. Choose appropriate relation types that describe how the topics are connected
5. Provide categories for topics when possible (method, technology, concept, field, etc.)
6. Include brief descriptions for complex topics
7. Return ONLY valid JSON that matches the provided schema structure

SCHEMA REQUIREMENTS:
- topics: Array of unique topics with label, category, and description
- edges: Array of relationships with sourceTopic, targetTopic, relation, weight, and description
- All topic labels referenced in edges must exist in the topics array
- Use descriptive relation names like 'uses', 'improves', 'compares', 'outperforms', 'represents'"""}

user_prompt = f"""
Analyze the following text and extract a knowledge graph with topics and their relationships.

STEPS:
1. Identify all key topics/entities in the text, with a category and brief description for each topic
2. Determine relationships between these topics
3. Create a JSON structure with:
   - topics: list of unique topics with labels, categories, and descriptions
   - edges: list of relationships between topics

TEXT TO ANALYZE:
{the_text}

Return only the JSON structure matching the provided schema."""

user_msg = {"role": "user", "content": user_prompt}

In [ ]:
response =  chat(
    messages=[system_msg, user_msg],
    model=LLM_model,
    format=KnowledgeGraph.model_json_schema(),
    options={'temperature': 0},)
#options={'temperature': 0},
#print(response.message.content)

{
  "topics": [
    {
      "label": "Standard Retrieval-Augmented Generation (RAG)",
      "description": "A method relying on chunk-based retrieval for information retrieval.",
      "category": "Method"
    },
    {
      "label": "chunk-based retrieval",
      "description": "The retrieval mechanism used by standard RAG.",
      "category": "Method"
    },
    {
      "label": "GraphRAG",
      "description": "An advancement over standard RAG using graph-based knowledge representation.",
      "category": "Method"
    },
    {
      "label": "Binary relations",
      "description": "Edges connecting only two entities, a limitation in existing graph-based RAG.",
      "category": "Concept"
    },
    {
      "label": "n-ary relations",
      "description": "Relations involving multiple entities (n >= 2) that binary relations struggle to represent.",
      "category": "Concept"
    },
    {
      "label": "HyperGraphRAG",
      "description": "A novel hypergraph-based RAG method repr

In [30]:
#print(response.message.content)
KG = KnowledgeGraph.model_validate_json(response.message.content)
print(KG.topics[0])

label='Standard Retrieval-Augmented Generation (RAG)' description='A method relying on chunk-based retrieval for information retrieval.' category='Method'


In [ ]:
def locator(text: str, topic: str) -> topicMap:
    locations = []
    start = 0
    while True:
        location = text.find(topic, start)
        if location == -1:
            break
        locations.append(location)
        start = location + len(topic)

    ting = topicMap(topic, locations)
    return ting

In [ ]:
x = locator(the_text, "Standard Retrieval-Augmented Generation (RAG)")
#print(x)
x 

In [ ]:
print(EdgeList.model_json_schema())

In [ ]:
topic_indexs = []
for edge in edge_list.edges:
    print(f"{edge.Source_Topic} --{edge.Relation}--> {edge.Target_Topic} (weight: {edge.Weight})")
    #print(f"Source_Topic '{edge.Source_Topic}' found at index: {location}")
    if edge.Source_Topic not in topic_indexs:
        topic_indexs.append(locator(the_text, edge.Source_Topic))
    if edge.Target_Topic not in topic_indexs:
        topic_indexs.append(locator(the_text, edge.Target_Topic))
    #print('-'*30)
    #break

topic_indexs
